In [1]:
import torch
from transformers import AutoModel,AutoTokenizer

/data/shishijie/anaconda3/envs/deberta/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
ptm_path='/home/shishijie/workspace/PTMs/deberta-base'
tokenizer=AutoTokenizer.from_pretrained(ptm_path)
model=AutoModel.from_pretrained(ptm_path)

In [16]:
text='你好'
tokenized_data=tokenizer(text,return_tensors='pt')
outputs=model(**tokenized_data,output_hidden_states=True)

In [25]:
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[ 0.0452, -0.0113, -0.0761,  ...,  0.0363,  0.0723, -0.0534],
         [-1.2469,  0.1110,  0.6634,  ...,  1.7498, -1.2915,  0.4342],
         [-0.1360, -0.8177,  0.6069,  ...,  2.2420, -0.6911,  0.5334],
         [ 0.5248,  0.2118,  1.0589,  ...,  1.5573, -1.1657,  0.4546],
         [ 0.2990, -0.2704,  0.4432,  ...,  2.2601, -0.6226,  0.5809],
         [ 0.2197,  0.0697, -0.1574,  ...,  0.0859,  0.1927,  0.0920]]],
       grad_fn=<AddBackward0>), hidden_states=(tensor([[[-0.3259,  0.3926, -0.5905,  ..., -0.2977, -0.1261,  0.2434],
         [-1.1396, -0.7813, -0.6273,  ...,  0.4993,  0.0726, -0.4027],
         [-1.2837, -0.6408,  0.4874,  ...,  1.8429,  0.5336,  1.1886],
         [-0.7672,  0.4678, -0.4241,  ...,  0.3074, -0.5668,  0.3768],
         [ 0.0948, -0.6262, -0.6328,  ..., -1.2387, -1.4035,  0.1604],
         [-0.3745,  0.7429, -0.1807,  ..., -0.4511, -0.5262,  0.4496]]],
       grad_fn=<MulBackward0>), tensor([[[-0.1885,  0.2611,  0.

In [27]:
print("Number of parameters:", sum(p.numel() for p in model.parameters()))

Number of parameters: 138601728


In [11]:
ten1=torch.tensor([[[1,2,3,4]],[[5,6,7,8]]])
ten2=torch.tensor([[[11,22,33,44]],[[55,66,77,88]]])
print(ten1)
ten=[]
ten+=ten1
ten+=ten2
print(ten)
ten=torch.cat(ten,dim=1)
print(ten)
ten=torch.sum(ten,dim=1)
ten

tensor([[[1, 2, 3, 4]],

        [[5, 6, 7, 8]]])
[tensor([[1, 2, 3, 4]]), tensor([[5, 6, 7, 8]]), tensor([[11, 22, 33, 44]]), tensor([[55, 66, 77, 88]])]
tensor([[ 1,  2,  3,  4,  5,  6,  7,  8, 11, 22, 33, 44, 55, 66, 77, 88]])


tensor([432])

In [1]:
import numpy as np
import json
from sklearn.metrics import roc_auc_score
# with open('./data/y_pred.json','r') as fr:
#     y_pred=json.loads(fr.read()) 

# with open('./data/y_true.json','r') as fr:
#     y_true=json.loads(fr.read()) 
    
y_true = np.array([[1, 0, 0, 0],
                   [0, 0, 0, 1],
                   [0, 1, 0, 0],
                   [0, 0, 0, 1],
                   [0, 0, 0, 1],
                   [0, 0, 0, 1]])
y_pred = np.array([[1, 0, 0, 0],
                    [0, 0, 0, 1],
                    [0, 0, 0, 1],
                    [1, 0, 0, 1],
                    [0, 0, 0, 1],
                    [0, 0, 0, 1]])
try:
    sc=roc_auc_score(y_true, y_pred) ## y_true=ground_truth
except ValueError:
    pass
print(sc)

NameError: name 'sc' is not defined

In [4]:
import json
with open('/home/shishijie/workspace/project/scholar/data/chatglm_input/homepage_test_with_label.json','r',encoding='utf8') as fr:
    data=json.loads(fr.read())

homepage_data=[]
for item in data:
    name=item['instruction'][11:-6]
    text=item['input'].strip().replace('\n','').replace('\r','').replace('\t','')
    if '不是个人主页' in item['output']:
        label='0'
    else:
        label='1'
    homepage_data.append(name+'[SEP]'+text+'\t'+label)

with open('./data/homepage.test','w',encoding='utf8') as fw:
    for item in homepage_data:
        fw.write(item+'\n')

In [11]:
import torch
from transformers import AutoTokenizer,AutoModel,AutoConfig
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm
# 数据处理
class MyDataset(Dataset):
    def __init__(self,data_path,ptm_path,max_length=512) -> None:
        super().__init__()
        self.tokenizer=AutoTokenizer.from_pretrained(ptm_path)
        
        self.data,self.label=self.get_data(data_path)
        self.label=torch.LongTensor(self.label)
        self.length=len(self.data)
        self.tokenized_data=self.tokenize_data(self.tokenizer,self.data,max_length)

    def get_data(self,path):
        data=[]
        label=[]
        with open(path,'r',encoding='utf8') as fr:
            for line in fr.readlines():
                line=line.strip().split('\t')
                data.append(line[0])
                label.append(int(line[1]))
        return data,label
    
    def tokenize_data(self,tokenizer,data,max_length):
        tokenizer_data=[]
        for item in tqdm(data,desc='Tokenizing'):
            tokenizer_data.append(
                tokenizer(item,truncation=True,padding='max_length',max_length=max_length,return_tensors='pt')
            )
        return tokenizer_data
    def __len__(self):
        return self.length
    
    def __getitem__(self, index) :
        input=self.tokenized_data[index]
        label=self.label[index]
        
        for k,v in input.items():
            input[k]=v.squeeze(0)
        
        
        return input,label

In [12]:
train_dataset=MyDataset('/home/shishijie/workspace/project/deberta_cls/data/gender.test','/home/shishijie/workspace/PTMs/albert-base-v2',128)

Tokenizing:   0%|          | 0/1831 [00:00<?, ?it/s]

Tokenizing: 100%|██████████| 1831/1831 [00:07<00:00, 245.96it/s]


In [19]:
type(train_dataset)

__main__.MyDataset

In [4]:
import json
from sklearn.metrics import classification_report
text2id={'气滞血瘀':0,'气阴两虚':1,'气虚血瘀':2,'寒凝血瘀':3,'心肾阳虚':4,'心血瘀阻':5,'心肾阴虚':6,'痰浊闭阻':7,'痰阻血瘀':8,'气滞心胸':9,'热毒血瘀':10,'寒凝心脉':11,'心气虚弱':12}
pred=[]
truth=[]
with open('/home/shishijie/workspace/project/ChatGLM-Efficient-Tuning/predict/diagnosis_train_5e-5_full/generated_predictions.jsonl','r',encoding='utf8') as fr:
    for item in fr.readlines():
        item=json.loads(item)
        pred.append(text2id[item['predict'][-4:]])
        truth.append(text2id[item['label'][-4:]])

report=classification_report(truth,pred)
print(report)

              precision    recall  f1-score   support

           0       0.74      0.81      0.77       334
           1       0.73      0.72      0.72       236
           2       0.59      0.63      0.61       213
           3       0.62      0.43      0.51        37
           4       0.80      0.50      0.62        16
           5       0.00      0.00      0.00         8
           6       1.00      0.14      0.25         7
           7       0.67      0.57      0.62         7
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00         3
          10       1.00      0.33      0.50         3
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00         1

    accuracy                           0.69       873
   macro avg       0.47      0.32      0.35       873
weighted avg       0.68      0.69      0.68       873



/data/shishijie/anaconda3/envs/deberta/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/shishijie/anaconda3/envs/deberta/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/shishijie/anaconda3/envs/deberta/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta